<a href="https://colab.research.google.com/github/Erssa001/ECGR_4105/blob/main/HW5_P2%263.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [310]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [311]:
file_path = '/content/drive/My Drive/2023-2024/ECGR_4105/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))

bi_vars =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
dataset[bi_vars] = dataset[bi_vars].apply(binary_map)
# Preprocessing

# uncomment line below for problem 2
#components_to_use = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
# uncomment line below for problem 3
components_to_use = ['area', 'bedrooms','bathrooms','stories','mainroad','guestroom','basement','hotwaterheating','airconditioning', 'parking', 'prefarea']
X = torch.tensor(dataset[components_to_use].values.copy())

scaler = StandardScaler()
dataset[num_vars] = scaler.fit_transform(dataset[num_vars])
X_n = torch.tensor(dataset[components_to_use].values.copy())

Y = torch.tensor(dataset.pop('price').values)

In [312]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

X_train = X[train_indices]
Y_train = Y[train_indices]

X_test = X[val_indices]
Y_test = Y[val_indices]

X_n_train = X_n[train_indices]
X_n_test = X_n[val_indices]

In [313]:
def model(X, *params):
  result = params[-1]
  for i in range(0, X.shape[1]):
    result = result + (X[:, i] * params[i])
  return result

def loss_fn(Y_pred, Y):
  squared_diffs = (Y_pred - Y)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, X_train, X_test, Y_train, Y_test):
  for epoch in range(1, n_epochs + 1):
    Y_pred_train = model(X_train, *params)

    train_loss = loss_fn(Y_pred_train, Y_train)

    with torch.no_grad(): # <1>
      Y_pred_test = model(X_test, *params)
      val_loss = loss_fn(Y_pred_test, Y_test)
      assert val_loss.requires_grad == False # <2>

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    bias_value = params[-1].item()
    if ((epoch == 1) or (epoch % 500 == 0)):
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f}, Validation loss {val_loss.item():.4f}")

  return params

In [314]:
params = torch.ones(X.shape[1] + 1, requires_grad=True)
with torch.no_grad():
  params[-1] = 0
learning_rate = .1
optimizer = optim.SGD([params], lr=learning_rate)

SGD_linear = training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    X_train = X_n_train, # <1>
    X_test = X_n_test, # <1>
    Y_train = Y_train,
    Y_test = Y_test)

params = torch.ones(X.shape[1] + 1, requires_grad=True)
with torch.no_grad():
  params[-1] = 0
learning_rate = 0.1
optimizer = optim.Adam([params], lr=learning_rate)

adam_linear = training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    X_train = X_n_train, # <1>
    X_test = X_n_test, # <1>
    Y_train = Y_train,
    Y_test = Y_test)

Epoch 1, Training loss 12.7202, Validation loss 11.4154
Epoch 500, Training loss 0.3366, Validation loss 0.2967
Epoch 1000, Training loss 0.3366, Validation loss 0.2965
Epoch 1500, Training loss 0.3366, Validation loss 0.2965
Epoch 2000, Training loss 0.3366, Validation loss 0.2965
Epoch 2500, Training loss 0.3366, Validation loss 0.2965
Epoch 3000, Training loss 0.3366, Validation loss 0.2965
Epoch 3500, Training loss 0.3366, Validation loss 0.2965
Epoch 4000, Training loss 0.3366, Validation loss 0.2965
Epoch 4500, Training loss 0.3366, Validation loss 0.2965
Epoch 5000, Training loss 0.3366, Validation loss 0.2965
Epoch 1, Training loss 12.7202, Validation loss 11.4154
Epoch 500, Training loss 0.3366, Validation loss 0.2965
Epoch 1000, Training loss 0.3366, Validation loss 0.2965
Epoch 1500, Training loss 0.3366, Validation loss 0.2965
Epoch 2000, Training loss 0.3366, Validation loss 0.2965
Epoch 2500, Training loss 0.3366, Validation loss 0.2965
Epoch 3000, Training loss 0.3366, V